In [2]:
synthetic_dir = '/nfs/students/winter-term-2020/project-2/YOLOv3-tiny/YOLOv3-tiny-chenrui/YOLOv3-tiny-chenrui-modularized/data/custom/images/valid/daytime_clear/'
batch_size = 1
height = 720
width = 1280

In [3]:
import argparse
import sys
import os
import json

import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch

from models import Generator, Discriminator
from datasets import ImageDataset

In [4]:
netG_A2B = Generator(3, 3)
netG_B2A = Generator(3, 3)
netG_A2B.cuda()
netG_B2A.cuda()
netG_A2B.load_state_dict(torch.load('output/netG_A2B.pth'))
netG_B2A.load_state_dict(torch.load('output/netG_B2A.pth'))
netG_A2B.eval()
netG_B2A.eval()

Generator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace=True)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace=True)
    (10): ResidualBlock(
      (conv_block): Sequential(
        (0): ReflectionPad2d((1, 1, 1, 1))
        (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
        (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace=True)
        (4): ReflectionPad2d((1, 1, 1, 1))
        (

In [5]:
netD_B = Discriminator(3)
netD_B.cuda()
netD_B.load_state_dict(torch.load('output/netD_B.pth'))
netD_B.eval()

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [6]:
Tensor = torch.cuda.FloatTensor
input_A = Tensor(batch_size, 3, height, width)
input_B = Tensor(batch_size, 3, height, width)

In [8]:
if not os.path.exists('datasets/synthetic/test/images'):
    os.makedirs('datasets/synthetic/test/images')
if not os.path.exists('datasets/synthetic/test/labels'):
    os.makedirs('datasets/synthetic/test/labels')

In [8]:
# sanity check: see if there are images that can't be opened for some reason
from PIL import Image

count = 0
invalid_imgs = []
for i in os.listdir(synthetic_dir):
    try: img = Image.open(synthetic_dir+i)
    except: 
        count += 1
        invalid_imgs.append(i)
        
assert(len(invalid_imgs)) == 0

In [10]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets

class SyntheticDataset(Dataset):

    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_list = list(set(os.listdir(root_dir)) - set(invalid_imgs))
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = os.path.join(self.root_dir, self.image_list[idx])
        img = Image.open(img_path)

        if self.transform:
            for t in self.transform:
                img = t(img)

        return img, self.image_list[idx]

In [11]:
transforms_ = [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) ]
dataloader = DataLoader(SyntheticDataset(synthetic_dir, transform=transforms_),
                            batch_size=batch_size, shuffle=False, num_workers=8)

In [14]:
D_values = {}

nbatch = 0
for daytime_images, daytime_image_filenames in dataloader:
    nbatch += batch_size

    # Set model input
    real_A = Variable(input_A.copy_(daytime_images))
    
    # Generate output
    fake_B = 0.5*(netG_A2B(real_A).data + 1.0) 
    batch_D_values = netD_B(fake_B).view(-1).tolist()

    for i in range(batch_size):
        D_values[daytime_image_filenames[i]] = batch_D_values[i]
        
        # Save weights in separate .txt files
        f = open('datasets/synthetic/valid/weights/' + daytime_image_filenames[i][:-4] + '.txt',"w")
        f.write(str(batch_D_values[i]))
        f.close()

        # Save image files
        save_image(fake_B[i], 'datasets/synthetic/valid/images/' + daytime_image_filenames[i])

    sys.stdout.write('\rGenerated images %04d of %04d' % (nbatch, len(dataloader)*batch_size))

sys.stdout.write('\n')

Generated images 1764 of 1764


In [15]:
import json

# Save weights in one .json file as well
with open('datasets/synthetic/valid/weights.json', 'w') as fp:
    json.dump(D_values, fp)